In [44]:
from learn_process import init_data, train_model
from networks.unet_leiterrl import weights_init, TurbNetG, UNet
from networks.dummy_network import DummyNet
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.nn import MSELoss
import torch.nn.functional as F

# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Initialize Data

In [48]:
 # init data
datasets_2D, dataloaders_2D = init_data(dataset_name="groundtruth_hps_no_hps/groundtruth_hps_overfit_10", reduce_to_2D=True, 
                                inputs="xyzt", labels="t", batch_size=3)

init done [total number of datapoints/runs: 10]


# Settings Training

In [49]:
# parameters of model and training
loss_fn = MSELoss()
n_epochs = 1000 #60000
lr=0.0001 #0.0004

#model = TurbNetG(channelExponent=4, in_channels=4, out_channels=2)
unet_model = UNet(in_channels=5, out_channels=1).float()
# TODO too many in channels for unet?
fc_model = DummyNet().float()
# model.to(device)

name_folder="try_unet_overfit10_batchsize3"

# Training

In [50]:
# train model
_, writer = train_model(unet_model, dataloaders_2D, loss_fn, n_epochs, lr, name_folder=name_folder)


epochs: 100%|██████████| 1000/1000 [10:09<00:00,  1.64it/s, loss: 0.4031]

Finished Training


In [51]:
writer = SummaryWriter(f"runs/{name_folder}")
for batch_idx, data_test in enumerate(dataloaders_2D["test"]):
    x = data_test.inputs.float()
    y = data_test.labels.float()
    y_out = unet_model(x)
    writer.add_image("x_unseen", x[0, 0, :, :], dataformats="WH")
    writer.add_image("y_unseen_out", y_out[0, 0, :, :], dataformats="WH")
    writer.add_image("y_unseen_true", y[0, 0, :, :], dataformats="WH")
            
# TODO vary lr, vary input_Vars
# TODO vary batch size
# TODO safe status (git)

# Visualization